<a href="https://colab.research.google.com/github/abhishek-paghdal/Content_Generation/blob/main/Copy_of_Antim_Bar222222.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 68.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
ERROR: pi

###**Required Importing**

In [3]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch

###**Uploading Dataset**

In [4]:
df=pd.read_csv('/content/drive/MyDrive/article_data3.csv')

In [5]:
df.shape

(184, 2)

In [6]:
df.head()

,Title,Content
0,GAME-CHANGING TRENDS TO LOOK OUT FOR WITH AI,"Apr 30, 2018\nArtificial Intelligence is a sta..."
1,Chatbots were the next big thing: what happened?,"The Startup\nJun 5, 2018\nOh, how the headline..."
2,502 Bad Gateway,No Content Found
3,Intuitively Understanding Convolutions for Dee...,Sign up\nSign In\nSign up\nSign In\nIrhum Shaf...
4,An intro to Machine Learning for designers,Sign up\nSign In\nSign up\nSign In\nUX Collect...


##**Preprocessing Dataset**

In [7]:
#Checking if NULL rows are present
df.isna().sum()

Title      0
Content    0
dtype: int64

In [8]:
#Removing the row with null values
df.dropna(inplace=True)

In [9]:
#Making new dataset 
df1=df[:180]

In [10]:
df1.shape

(180, 2)

In [11]:
df1.to_csv('data1.csv',index=False)

In [12]:
data=pd.read_csv('data1.csv')

In [16]:
data.shape

(124, 2)

In [14]:
#Removing duplicates rows
data.drop_duplicates(inplace=True)

In [15]:
data.shape

(124, 2)

####**Now check the language of texts**

In [16]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 48.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993243 sha256=29c7d918ade9c4f5bcec430500a162a8ef2dbec99e55a8563fbb550d2748747f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [17]:
import langdetect

In [18]:
def is_english(text):
    try:
        lang = langdetect.detect(text)
        if lang == 'en':
            return True
        else:
            return False
    except:
        return False

In [20]:
#data['is_english'] = data['Content'].apply(lambda x: is_english(x))

In [17]:
data.head()

,Title,Content
0,GAME-CHANGING TRENDS TO LOOK OUT FOR WITH AI,"Apr 30, 2018\nArtificial Intelligence is a sta..."
1,Chatbots were the next big thing: what happened?,"The Startup\nJun 5, 2018\nOh, how the headline..."
2,502 Bad Gateway,No Content Found
3,Intuitively Understanding Convolutions for Dee...,Sign up\nSign In\nSign up\nSign In\nIrhum Shaf...
4,An intro to Machine Learning for designers,Sign up\nSign In\nSign up\nSign In\nUX Collect...


In [18]:
#data['is_english'].value_counts()

In [23]:
#data= data[data['is_english']==True]

###**Text Length Count**

In [14]:
from collections import Counter

In [15]:
data['text_length'] = data['Content'].apply(len)

In [16]:
data.sort_values(by='text_length')

,Title,Content,text_length
2,502 Bad Gateway,No Content Found,16
17,No Title Found,Sign up\nSign In\nSign up\nSign In\n,32
23,5 Cutting-Edge Technologies That Will Transfor...,Get hands-on learning from ML experts on Cours...,51
25,Why AI Will Bring an Explosion of New Jobs,\nReceive Stories from @daniel-jeffries\nGet h...,90
8,Did Google Duplex just pass the Turing Test?,"May 8, 2018\nMember-only\nI think it was the f...",618
...,...,...,...
35,The Birth of Wetware,Sign up\nSign In\nSign up\nSign In\nproto.life...,22417
43,Nature 2.0,Sign up\nSign In\nSign up\nSign In\nOcean Prot...,24729
19,Artificial Intelligence — The Revolution Hasn’...,"Apr 19, 2018\nArtificial Intelligence (AI) is ...",24800
157,Multi-Modal Methods: Image Captioning (From Tr...,Sign up\nSign In\nSign up\nSign In\nML Review\...,26511


####**Now consider articles with length between (300 and 30000)**

In [21]:
data= data[(data['text_length'] >= 300) & (data['text_length'] <= 300000)]

In [22]:
data.shape

(113, 3)

##**Split the dataset into train and evaluate**

In [19]:
train_data, eval_data = train_test_split(data, test_size=0.2)

In [20]:
train_data.shape

(99, 2)

In [21]:
train_data.to_csv('train_data.csv')
eval_data.to_csv('eval_data.csv')

In [22]:
# this is for the train data

# Open the CSV file and read the contents
with open('/content/train_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['Content'] for row in reader]

# Write the contents to a text file
with open('/content/train_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [23]:
# this is for the evaluate data

# Open the CSV file and read the contents
with open('/content/eval_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['Content'] for row in reader]

# Write the contents to a text file
with open('/content/eval_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


##**Load the model and tokenizer**

In [ ]:
# # Load the pre-trained GPT-2 tokenizer and model
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2LMHeadModel.from_pretrained('gpt2')

In [24]:
#import torch
#from instruct_pipeline import InstructionTextGenerationPipeline
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-3b", device_map="auto", torch_dtype=torch.bfloat16)

#generate_text = InstructionTextGenerationPipeline(model=model, tokenizer=tokenizer)


In [25]:
# Set requires_grad=True for all parameters in the model
for param in model.parameters():
    param.requires_grad = True

# Freeze all layers except the last two
for name, param in model.named_parameters():
    if 'layers.30' not in name and 'layers.31' not in name:# Only unfreeze the last two 
       
        param.requires_grad = False
    print(name, param.requires_grad) 

gpt_neox.embed_in.weight False
gpt_neox.layers.0.input_layernorm.weight False
gpt_neox.layers.0.input_layernorm.bias False
gpt_neox.layers.0.post_attention_layernorm.weight False
gpt_neox.layers.0.post_attention_layernorm.bias False
gpt_neox.layers.0.attention.query_key_value.weight False
gpt_neox.layers.0.attention.query_key_value.bias False
gpt_neox.layers.0.attention.dense.weight False
gpt_neox.layers.0.attention.dense.bias False
gpt_neox.layers.0.mlp.dense_h_to_4h.weight False
gpt_neox.layers.0.mlp.dense_h_to_4h.bias False
gpt_neox.layers.0.mlp.dense_4h_to_h.weight False
gpt_neox.layers.0.mlp.dense_4h_to_h.bias False
gpt_neox.layers.1.input_layernorm.weight False
gpt_neox.layers.1.input_layernorm.bias False
gpt_neox.layers.1.post_attention_layernorm.weight False
gpt_neox.layers.1.post_attention_layernorm.bias False
gpt_neox.layers.1.attention.query_key_value.weight False
gpt_neox.layers.1.attention.query_key_value.bias False
gpt_neox.layers.1.attention.dense.weight False
gpt_neox.l

##**Prepare your dataset by creating a TextDataset object**

In [26]:
# Prepare your dataset by creating a TextDataset object
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/train_dataset.txt',
    block_size=128
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [27]:
# Prepare your dataset by creating a TextDataset object
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/eval_dataset.txt',
    block_size=128
)


##**checkpoint using WANDB**

In [ ]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import wandb
from transformers import Trainer, TrainingArguments
from transformers.integrations import WandbCallback

In [ ]:
# Set up WANB
wandb.login()
wandb.init(project="my_project_tech_articles")


wandb: Currently logged in as: pathakabhishek978 (lazy_coders). Use `wandb login --relogin` to force relogin


##**Set up the trainer with the appropriate training arguments**

In [28]:


# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    num_train_epochs=1,
    save_total_limit=2,
    save_steps=62,
    resume_from_checkpoint=True,
   # report_to='wandb',  # Enable WandbCallback
   # load_best_model_at_end=True  # load best model checkpoint at the end
)

# Define the WandbCallback
#wandb_callback = WandbCallback()

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    #callbacks=[WandbCallback()],
)

##**Train the model**

In [41]:
torch.cuda.empty_cache()

In [29]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.931600,2.937813


TrainOutput(global_step=1684, training_loss=2.9835097795427554, metrics={'train_runtime': 1419.0149, 'train_samples_per_second': 1.187, 'train_steps_per_second': 1.187, 'total_flos': 3422581146255360.0, 'train_loss': 2.9835097795427554, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 3.3943207263946533,
 'eval_runtime': 7.0943,
 'eval_samples_per_second': 95.429,
 'eval_steps_per_second': 11.981,
 'epoch': 5.0}

In [ ]:
# # Save the best model checkpoint
 trainer.save_model('/path/to/save/directory')


###**Now code to rehit the promt to get required length article**

In [ ]:

# Set the maximum length of each segment
max_segment_length = 300

# Set the initial prompt
prompt = "Artificial intelligence"

# Initialize the generated text
generated_text = prompt

# Generate segments of text and stitch them together
while len(generated_text) < 500:
    # Encode the prompt as input IDs
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    # Generate the next segment of text
    output = model.generate(input_ids=input_ids.to('cuda'),
                             max_length=max_segment_length,
                             do_sample=True,
                             top_p=0.95,
                             temperature=0.7,
                             top_k=50)
    
    # Decode the output to text
    segment = tokenizer.decode(output[0], skip_special_tokens=True)

    # Add the segment to the generated text
    generated_text += segment

    # Get the last 1024 tokens of the previous segment as the new prompt
    prompt = generated_text[-max_segment_length:]

# Print the generated text
print(generated_text)
final_text=generated_text


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial intelligenceArtificial intelligence (AI) technology is already in the works, and it’s not just the companies that are looking for a new way to do it. The technology is already in its infancy, but the biggest growth is expected in the next few years, when AI is expected to become a dominant driver of future industries. Many companies are also looking for ways to make AI more accessible, by offering a set of tools for developers to integrate into their own systems. The new tools are being made available to developers through the “Digital Machines” platform, which is open source and accessible to the public. Developers can “use the Digital Machines” platform to connect with a number of existing and emerging AI capabilities, and they can create AI software that can be integrated into existing software and applications.” The new tools will be available to developers through the “Digital Machines” platform, which is open source and accessible to the public. Developers can also cre

In [ ]:
print(len(final_text))

1592


In [ ]:
type(final_text)

str

##**Use text summarization to improve article**

In [ ]:
# Install required libraries
#!pip install transformers

# Import required libraries
from transformers import pipeline

# Load pre-trained summarization model
#summarizer = pipeline("summarization")
# Load pre-trained summarization model (using BART-Large-CNN)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


# Define input text to summarize
input_text = "The United States is a country located in North America. It is the third largest country in the world by total area and the third most populous country in the world. The United States has a diverse geography and climate, with mountains, forests, deserts, and coastal areas. Its economy is the largest in the world, driven by industries such as technology, finance, and healthcare."

# Generate summary for input text
summary = summarizer(final_text, max_length=1000, min_length=30, do_sample=False)

# Print generated summary
print(summary[0]['summary_text'])


Your max_length is set to 1000, but you input_length is only 304. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=152)


New tools are being made available to developers through the “Digital Machines’ platform, which is open source and accessible to the public. Developers can “use the Digital Machines” platform to connect with a number of existing and emerging AI capabilities.


##**Below code is for save the model**

In [ ]:
# model.to('cuda')a# Convert the model to CPU

model = model.to(torch.device('cpu'))


In [48]:

# Generate text using the fine-tuned model
prompt ='what is artificial intelligence'
generated_text = model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt'),
    max_length=200,
    do_sample=True,
    top_k=100,
    top_p=0.95,
    temperature=0.7,
)
final_text=tokenizer.decode(generated_text[0], skip_special_tokens=True)
print(final_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


what is artificial intelligence?
--Artificial intelligence is a branch of computer science that is about making machines that are as intelligent as the humans. Artificial intelligence is very powerful, and it can make many decisions. There are computers that are very powerful and they have the ability to learn. These learning computers are called Deep Learning Computers. They can figure out a pattern by looking at many examples of the pattern and try to figure out the pattern by themselves. Deep Learning Computers can help us in many ways. You can ask it a question and it can figure out the answer. You can teach a computer to play a game. It can beat all the humans in the world at the game. You can also use the artificial intelligence for the good. You can use artificial intelligence to help you solve problems. With the artificial intelligence we can improve the environment and we can make it safer. For example, we can make a robot that can help out in case of an accident in a factory.

In [41]:
# Install required libraries
#!pip install transformers

# Import required libraries
from transformers import pipeline

# Load pre-trained summarization model
#summarizer = pipeline("summarization")
# Load pre-trained summarization model (using BART-Large-CNN)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


# Define input text to summarize
#input_text = "The United States is a country located in North America. It is the third largest country in the world by total area and the third most populous country in the world. The United States has a diverse geography and climate, with mountains, forests, deserts, and coastal areas. Its economy is the largest in the world, driven by industries such as technology, finance, and healthcare."

# Generate summary for input text
summary = summarizer(final_text, max_length=200, min_length=30, do_sample=False)

# Print generated summary
print(summary[0]['summary_text'])


 AI has been around for decades, but in recent years, it has made huge strides in the field of computer vision and natural language processing. With the help of these technologies, computers can now recognize faces, parse sentences, and perform many other tasks.


In [49]:
# trainer.ave_model('Second_Model_Dooly-Epoch_3_1000')

In [ ]:
# import zipfile
# zip_ref = zipfile.ZipFile("Model_2_GPT-2_Epoch_3_1000.zip", "r")
# zip_ref.extractall()
# zip_ref.close()

In [50]:
import joblib

# joblib.dump(model, 'SecondModel_Epoch_3_1000_2.joblib')/content/drive/MyDrive/Colab Notebooks

In [51]:
joblib.dump(model,'model_dolly_v2_1_epoch_1.joblib')
joblib.dump(tokenizer,'tokenizer_dolly_v2_1_epoch_1.joblib')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/joblib/numpy_pickle.py:553 in dump                       │
│                                                                                                  │
│   550 │   │   │   NumpyPickler(f, protocol=protocol).dump(value)                                 │
│   551 │   elif is_filename:                                                                      │
│   552 │   │   with open(filename, 'wb') as f:                                                    │
│ ❱ 553 │   │   │   NumpyPickler(f, protocol=protocol).dump(value)                                 │
│   554 │   else:                                                                                  │
│   555 │   │   NumpyPickler(filename, protocol=protocol).dump(value)                              │
│   556                                                                                            │
│                                                                                                  │
│ /usr/lib/python3.10/pickle.py:487 in dump                                                        │
│                                                                                                  │
│    484 │   │   │   self.write(PROTO + pack("<B", self.proto))                                    │
│    485 │   │   if self.proto >= 4:                                                               │
│    486 │   │   │   self.framer.start_framing()                                                   │
│ ❱  487 │   │   self.save(obj)                                                                    │
│    488 │   │   self.write(STOP)                                                                  │
│    489 │   │   self.framer.end_framing()                                                         │
│    490                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/joblib/numpy_pickle.py:355 in save                       │
│                                                                                                  │
│   352 │   │   │   wrapper.write_array(obj, self)                                                 │
│   353 │   │   │   return                                                                         │
│   354 │   │                                                                                      │
│ ❱ 355 │   │   return Pickler.save(self, obj)                                                     │
│   356                                                                                            │
│   357                                                                                            │
│   358 class NumpyUnpickler(Unpickler):                                                           │
│                                                                                                  │
│ /usr/lib/python3.10/pickle.py:603 in save                                                        │
│                                                                                                  │
│    600 │   │   │   │   │   │   │   │   "two to six elements" % reduce)                           │
│    601 │   │                                                                                     │
│    602 │   │   # Save the reduce() output and finally memoize the object                         │
│ ❱  603 │   │   self.save_reduce(obj=obj, *rv)                                                    │
│    604 │                                                   

In [ ]:


joblib.dump(model, 'Model_Epoch_5_3000.joblib')

['Model_Epoch_5_3000.joblib']

In [ ]:
joblib.dump(tokenizer, 'tokenizer_5_3000.joblib')


['tokenizer_5_3000.joblib']

In [ ]:
loaded_model_joblib=joblib.load('SecondModel_Epoch_3_1000_2.joblib')

FileNotFoundError: ignored

In [ ]:
loaded_model_joblib

In [ ]:
loaded_model_joblib

In [ ]:
import joblib as jb

In [ ]:
jb.dump(model,'model_jb.pkl')

In [ ]:
loaded_model = jb.load('model_jb.pkl')